In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import researchpy_fork as rp # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import specification_curve_fork as specy # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

# Analysis plan:

1. ## [Descriptives and tables](./1.%20descriptives_and_tables.ipynb)
2. ## [Visualization](./2.%20visualization.ipynb)
3. ## [Frequencies and Normality tests](./2.%20frequencies_and_normality_test.ipynb)
   1. ### Frequencies, histograms, and QQ plots
      * Normal test
      * Kurtosis test
      * Shapiro
      * Anderson
      * Bartlett
   2. ### Correlation between independent variables (IVs) and control variables and Multicolinarity test
      * Pearson's R
      * VIF
     - ***ivs_dummy*** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
     - ***% Sector per Workforce*** (continous ratio) = Sector percentage per worksforce (0-100)
     - ***num_words*** (continous ratio) = Number of words in job description
     - ***English Requirement in Job Ad*** (binary nominal) = English requirement in job description (0 vs. 1)
     - ***Dutch Requirement in Job Ad*** (binary nominal) = Dutch requirement in job description (0 vs. 1)
     - ***Platform*** (binary dummy) = LinkedIn (0 vs. 1), Indeed (0 vs. 1), Glassdoor (0 vs. 1)

4. ## [ANOVA and Chi-square (Pearson's R)](./3.%20chisqt_and_anova.ipynb)

   1. ### Chi-square
      * **df_manual:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs*** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
      * **df_jobs:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs*** (binary nominal) = Social category designation (Female, Male, Mixed Gender)

   2. ### One-way ANOVA, interactions, and post-hoc test
      * **df_manual:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs*** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
          - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
          - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
      * **df_jobs:**
         - ***dvs_prob*** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
         - ***ivs*** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
           - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
           - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test

5. ## [Regression Analysis](./3.%20regression_analysis.ipynb)
   1. ### Logistic Regression  with all interaction (smf):
      * **df_manual:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
      * **df_jobs:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
   2. ### OLS Regression with all interaction:
      * **df_jobs:**
        - ***dvs_prob*** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
   3. ### Multilevel OLS Regression with all interaction:
      * **df_jobs:**
        - ***dvs_prob*** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)

6. ## [Specification Curve Analysis](./4.%20specification_curve_analysis.ipynb)

   1. ### Logistic Specification Curve Analysis:
      * **df_manual:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
      * **df_jobs:**
        - ***dvs*** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)
   2. ### OLS Specification Curve Analysis:
      * **df_jobs:**
        - ***dvs_prob*** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - ***ivs_perc*** (continous ratio) = Social category percentage per sector (0-100)


# READ DATA

In [3]:
with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_training.pkl')
assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_manual.shape}')
df_manual = categorize_df_gender_age(df_manual)


Dataframe loaded with shape: (5947, 75)


In [4]:
with open(f'{data_dir}df_jobs_for_analysis_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
print(f'Dataframe loaded with shape: {df_jobs.shape}')
df_jobs = categorize_df_gender_age(df_jobs)


Dataframe loaded with shape: (308583, 101)


## Set dataframes

#### Set variables

In [5]:
# Dataframes dict
dataframes = {
    'df_jobs': df_jobs,
    # 'df_manual': df_manual,
}

# Models dict
sm_models = {
    'OLS': sm.OLS,
    'Logistic': sm.Logit,
}

# DVs dict for analysis
dvs_for_analysis = {
    'probability': ['Probability Warmth and Competence', dvs_prob],
    'binary': ['Categorical Warmth and Competence', dvs],
    'binary and probability': ['Categorical and Probability Warmth and Competence', dvs_all],
}

# Make extra IV dicts
ivs_dummy_for_analysis = [iv for iv in ivs_dummy if 'Mixed' not in iv]
ivs_dummy_and_perc_for_analysis = [iv for iv in ivs_dummy_and_perc if 'Mixed' not in iv]
ivs_dummy_perc_and_perc_interactions_for_analysis = [iv for iv in ivs_dummy_perc_and_perc_interactions if 'Mixed' not in iv]


# IVs dict for analysis
ivs_for_analysis = {
    # 'categories, percentages, and interactions': [
    #     'Categorical, PPS, and PPS Interactions Gender and Age',
    #     ivs_dummy_perc_and_perc_interactions_for_analysis
    # ],
    'categories and percentages': [
        'Categorical and PPS Gender and Age',
        ivs_dummy_and_perc_for_analysis
    ],
    'percentages and interactions': [
        'PPS and PPS Interactions',
        ivs_perc_and_perc_interactions
    ],
    'categories': [
        'Categorical Gender and Age',
        ivs_dummy_for_analysis
    ],
    'percentages': [
        'PPS Gender and Age',
        ivs_perc
    ],
    'interactions': [
        'PPS Interactions',
        ivs_perc_interactions
    ],
}


# Functions

In [6]:
# Dataframes with '%' and spaces removed and replaced with '_'
def change_vars_for_formula(df, print_enabled=True):
    df = df.copy().rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_jobs.columns})

    # Variable names for statsmodels regression formulas with '%' and spaces removed and replaced with '_'
    ivs_perc_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc))
    ivs_perc_interactions_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc_interactions))
    ivs_dummy_and_perc_for_analysis_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_dummy_and_perc_for_analysis))
    ivs_dummy_perc_and_perc_interactions_for_analysis_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_dummy_perc_and_perc_interactions_for_analysis))
    ivs_perc_and_perc_interactions_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc_and_perc_interactions))
    controls_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), controls))
    controls_for_formula = ' + '.join(controls_[:1])

    if print_enabled:
        print('-'*20)
        print(f'IVs perc to use:\n{ivs_perc_}')
        print('\n')
        print('-'*20)
        print(f'IVs perc interactions to use:\n{ivs_perc_interactions_}')
        print('\n')
        print('-'*20)
        print(f'IVs dummy and perc to use:\n{ivs_dummy_and_perc_for_analysis_}')
        print('\n')
        print('-'*20)
        print(f'IVs dummy, perc, and perc interactions to use:\n{ivs_dummy_perc_and_perc_interactions_for_analysis_}')
        print('\n')
        print('-'*20)
        print(f'IVs perc and perc interactions to use:\n{ivs_perc_and_perc_interactions_}')
        print('\n')
        print('-'*20)
        print(f'All controls:\n{controls_}')
        print('\n')
        print('-'*20)
        print(f'Controls to use:\n{controls_for_formula}')
        print('\n')


    # IVs dict for analysis
    ivs_for_analysis_ = {
        'categories, percentages, and interactions': [
            'Categorical, PPS, and PPS Interactions Gender and Age',
            ivs_dummy_perc_and_perc_interactions_for_analysis_
        ],
        'categories and percentages': [
            'Categorical and PPS Gender and Age',
            ivs_dummy_and_perc_for_analysis_
        ],
        'percentages and interactions': [
            'PPS and PPS Interactions',
            ivs_perc_and_perc_interactions_
        ],
        'categories': [
            'Categorical Gender and Age',
            ivs_dummy_for_analysis
        ],
        'percentages': [
            'PPS Gender and Age',
            ivs_perc_
        ],
        'interactions': [
            'PPS Interactions',
            ivs_perc_interactions_
        ],
    }

    return df, ivs_for_analysis_, controls_, controls_for_formula


In [7]:
def save_df_full_summary_excel(
    df_full_summary,
    title,
    text_to_add_list,
    file_save_path,
    sheet_name=None,
    startrow=None,
    startcol=None,
):
    if sheet_name is None:
        sheet_name = 'All'
    if startrow is None:
        startrow = 1
    if startcol is None:
        startcol = 1

    # Define last rows and cols locs
    header_range = 1
    endrow = startrow + header_range + df_full_summary.shape[0]
    endcol = startcol + df_full_summary.shape[1]

    # Remove NAs
    df_full_summary = df_full_summary.fillna('')

    # Write
    writer = pd.ExcelWriter(f'{file_save_path}.xlsx')
    df_full_summary.to_excel(writer, sheet_name=sheet_name, merge_cells=True, startrow=startrow, startcol=startcol)
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    worksheet.set_column(startrow, 1, None, None, {'hidden': True}) # hide the index column

    # Title
    worksheet.merge_range(1, startcol, 1, endcol, title, workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'left', 'top': True, 'bottom': True, 'left': False, 'right': False}))

    # Main body
    body_max_row_idx, body_max_col_idx = df_full_summary.shape

    for c, r in tqdm_product(range(body_max_col_idx), range(body_max_row_idx)):
        row_to_write = startrow + header_range + r
        col_to_write = startcol + 1 + c # 1 is for index
        body_formats = {'num_format': '0.00', 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'center', 'text_wrap': True, 'left': False, 'right': False}

        if r == 0:
            body_formats |= {'top': True, 'bottom': True, 'left': False, 'right': False}
            worksheet.set_column(col_to_write, col_to_write, 10)

        if r == body_max_row_idx-1:
            body_formats |= {'bottom': True}

        if c == 0:
            body_formats |= {'align': 'left'}
            worksheet.set_column(col_to_write, col_to_write, 15)

        worksheet.write(row_to_write, col_to_write, df_full_summary.iloc[r, c], workbook.add_format(body_formats))

    # Add Note
    note_format = {'italic': True, 'font_name': 'Times New Roman', 'font_size': 10, 'font_color': 'black', 'align': 'left', 'left': False, 'right': False}
    worksheet.merge_range(endrow, startcol, endrow, endcol, 'Note.', workbook.add_format(note_format))
    # Add text
    for i, text in enumerate(text_to_add_list):
        worksheet.merge_range(endrow + 1 + i , startcol, endrow + 1 + i, endcol, text, workbook.add_format(note_format))

    writer.close()


In [8]:
def make_full_report(
    results, dv, analysis_type, model_name, dvs_name, ivs_name, ivs_type, df_name,
    regression_info_dict=None, regressor_order=None, text_to_add_list=None, title=None, model_names=None
):
    '''
    Make a full report for a regression analysis.
    results: statsmodels regression results object or list of results objects
    dv: str, dependent variable name
    '''

    if regression_info_dict is None:
        # Regression info dict
        regression_info_dict = {
            'F': lambda x: f'{x.fvalue:.2f}',
            'F (p-value)': lambda x: f'{x.f_pvalue:.2f}',
            'df_model': lambda x: f'{x.df_model:.0f}',
            'df_resid': lambda x: f'{x.df_resid:.0f}',
            'df_total': lambda x: f'{x.df_resid + x.df_model + 1:.0f}',
            'R-squared': lambda x: f'{x.rsquared:.2f}',
            'R-squared Adj.': lambda x: f'{x.rsquared_adj:.2f}',
            'Unstandardized Coefficent B (b)': lambda x: f'{x.params[0]:.2f}',
            'Standard Error (SE)': lambda x: f'{x.bse[0]:.2f}',
            'Standardized Coefficient b* (β)': lambda x: f'{x.params[0] / x.model.endog.std():.2f}',
            't': lambda x: f'{x.tvalues[0]:.2f}',
            't (p-value)': lambda x: f'{x.pvalues[0]:.2f}',
            '95% CI': lambda x: f'{x.conf_int().iloc[0, 1]:.2f} - {x.conf_int().iloc[0, 1]:.2f}',
            'Log-Likelihood': lambda x: f'{x.llf:.2f}',
            'Pseudo R2': lambda x: f'{x.prsquared:.2f}',
            'AIC': lambda x: f'{x.aic:.2f}',
            'BIC': lambda x: f'{x.bic:.2f}',
            'ICC': lambda x: f'{x.rsquared / (x.rsquared + (x.nobs - 1) * x.mse_resid):.2f}',
            'RMSE': lambda x: f'{x.mse_resid ** 0.5:.2f}',
            'RMSE (std)': lambda x: f'{x.mse_resid ** 0.5 / x.model.endog.std():.2f}',
            'Omnibus': lambda x: f'{sms.omni_normtest(x.resid).statistic:.2f}',
            'Omnibus (p-value)': lambda x: f'{sms.omni_normtest(x.resid).pvalue:.2f}',
            'Skew': lambda x: f'{sms.jarque_bera(x.resid)[-2]:.2f}',
            'Kurtosis': lambda x: f'{sms.jarque_bera(x.resid)[-1]:.2f}',
            'Jarque-Bera (JB)': lambda x: f'{sms.jarque_bera(x.resid)[0]:.2f}',
            'Jarque-Bera (p-value)': lambda x: f'{sms.jarque_bera(x.resid)[1]:.2f}',
            'Model Name': lambda x: f'{x.model.__class__.__name__}',
            'N': lambda x: f'{int(x.nobs):d}',
            # 'Summary': lambda x: f'{x.summary()}',
            # 'F (p-value - FDR)': lambda x: f'{x.f_pvalue_fdr:.2f}',
            # 'F (p-value - Bonferroni)': lambda x: f'{x.f_pvalue_bonf:.2f}',
            # 't (p-value - FDR)': lambda x: f'{x.pvalues_fdr[1]:.2f}',
            # 't (p-value - Bonferroni)': lambda x: f'{x.pvalues_bonf[1]:.2f}',
        }
        if isinstance(results, list):
            results_to_check = results[0]
        else:
            results_to_check = results
        if all('const' in x for x in zip(results_to_check.params.index, results_to_check.bse.index, results_to_check.tvalues.index, results_to_check.pvalues.index)):
            regression_info_dict = regression_info_dict | {
                'Intercept': lambda x: f'{x.params["const"]:.5f}',
                'Intercept (std)': lambda x: f'{x.bse["const"]:.5f}',
                'Intercept t': lambda x: f'{x.tvalues["const"]:.5f}',
                'Intercept t (p-value)': lambda x: f'{x.pvalues["const"]:.5f}',
                'Intercept (95% CI)': lambda x: f'{x.conf_int().loc["const"][0]:.5f} - {x.conf_int().loc["const"][1]:.5f}',
            }
    if model_names is None:
        if isinstance(results, list):
            model_names = [
                f'{results_to_check.model.endog_names.split("_")[0] if "_" in results_to_check.model.endog_names else results_to_check.model.endog_names} Model {i}'
                for i in range(len(results))
            ]
            model_names[0] = model_names[0].replace('Model 0', 'Full Model')
        else:
            model_names = [
                f'{results.model.endog_names.split("_")[0] if "_" in results.model.endog_names else results.model.endog_names}'
            ]

    order_type = 'unordered' if regressor_order is None else 'ordered'
    if text_to_add_list is None:
        text_to_add_list = []
        if regressor_order is not None:
            text_to_add_list.append('Models are ordered by independent variable type.')

        else:
            text_to_add_list.append('Models are ordered by coefficient size, largest to smallest.')

    if title is None:
        title = f'{model_name} {analysis_type}: {dvs_name} x {ivs_name}'

    try:
        # Statsmodels summary_col
        full_summary = summary_col(
            results,
            stars=True,
            info_dict=regression_info_dict,
            regressor_order=regressor_order,
            float_format='%0.2f',
            model_names=model_names,
        )
        if isinstance(results, list) and len(results) > 4:
            full_summary.tables[0][full_summary.tables[0].filter(regex='Full Model').columns[0]].loc['Unstandardized Coefficent B (b)': '95% CI'] = ''

        # Add title and notes
        full_summary.add_title(title)
        text_to_add_list.extend(full_summary.extra_txt)
        for text in text_to_add_list:
            full_summary.add_text(text)
        # Save
        save_name = f'{table_save_path}{model_name} {df_name} - ALL {dv} {order_type} {analysis_type} on {ivs_type}'
        df_full_summary = pd.read_html(full_summary.as_html())[0]
        df_full_summary.to_csv(f'{save_name}.csv')
        df_full_summary.style.to_latex(f'{save_name}.tex', hrules=True)
        save_df_full_summary_excel(df_full_summary, title, text_to_add_list, save_name)

        return full_summary
    except IndexError as e:
        print(f'Making full report for {model_names[0]} due to the following error: {e}')
        return None


In [9]:
def get_standardized_coefficients(results):

    # # Get standardized regression coefficients
    # std = np.asarray(constant.std(0))

    # if 'const' in results.params and 'const' in constant:
    #     std[0] = 1
    # tt = results.t_test(np.diag(std))
    # tt.c_names = results.model.exog_names

    # t-test
    std = results.model.exog.std(0)
    if 'const' in results.params:
        std[0] = 1
    tt = results.t_test(np.diag(std))
    if results.model.__class__.__name__ == 'MixedLM' or 'Group Var' in results.model.exog_names:
        offset = slice(None, -1)
        tt.c_names = results.model.exog_names[offset]
    else:
        offset = slice(None, None)
        tt.c_names = results.model.exog_names

    # Make df with standardized and unstandardized coefficients
    df_std_coef = pd.DataFrame(
        {
            'coef': results.params[offset].apply(lambda x: f'{x:.5f}'),
            'std err': results.bse[offset].apply(lambda x: f'{x:.5f}'),
            'std coef': (results.params[offset] / results.model.exog[offset].std(axis=0)).apply(lambda x: f'{x:.5f}'),
            't': results.tvalues[offset].apply(lambda x: f'{x:.5f}'),
            'P>|t|': results.pvalues[offset].apply(lambda x: f'{x:.5f}'),
            '[0.025': results.conf_int()[0][offset].apply(lambda x: f'{x:.5f}'),
            '0.975]': results.conf_int()[1][offset].apply(lambda x: f'{x:.5f}'),
        }
    )
    # if 'Group Var' in df_std_coef.index:
    #     df_std_coef = df_std_coef.drop('Group Var', axis='index')
    # # Add standardized coefficients and other data from t-test
    # df_std_coef['std coef'] = tt.effect
    # df_std_coef['std err'] = tt.sd
    # df_std_coef['t'] = tt.statistic
    # df_std_coef['P>|t|'] = tt.pvalue
    # df_std_coef['[0.025'] = tt.conf_int()[:, 0]
    # df_std_coef['0.975]'] = tt.conf_int()[:, 1]
    # df_std_coef['var'] = [names[i] for i in range(len(results.model.exog_names))]
    # df_std_coef = df_std_coef.sort_values('std coef', ascending=False)
    df_std_coef = df_std_coef.reset_index().rename(columns={'index': 'var'})
    df_std_coef = df_std_coef.rename(
        columns={
            'var': 'Variable',
            'coef': 'Unstandardized Coefficent B (b)',
            'std err': 'Standard Error',
            'std coef':'Standardized Coefficient b* (β)',
            't': 't-value',
            'P>|t|': 'p-value',
            '[0.025': '95% CI Lower',
            '0.975]': '95% CI Upper'
        }
    )
    # Reorder columns
    df_std_coef = df_std_coef[[
        'Variable',
        'Unstandardized Coefficent B (b)',
        'Standard Error',
        'Standardized Coefficient b* (β)',
        't-value',
        'p-value',
        '95% CI Lower',
        '95% CI Upper'
    ]]

    return tt, df_std_coef


In [10]:
def get_multilevel_reg_data(results, endog, groups, exog_restricted=None):
    '''
    perform likelihood ratio test of random-effects (LRT)
    + Degrees of Freedom (df)
    + Pseudo R-squared (pseudo_r2)
    + Intraclass Correlation Coefficient (ICC)
    + Bayesian Information Criterion (BIC)
    + Akaike Information Criterion (AIC)
    '''

    if exog_restricted is None:
        exog_names = results.params.index[:-1].tolist()
        exog_restricted = np.zeros((len(endog), 1))
        exog_restricted[1:, :] = 1

    # Null model
    null_model = sm.MixedLM(endog=endog, exog=exog_restricted, groups=groups)
    null_results = null_model.fit(reml=False)

    # LRT
    lrt = np.abs(null_results.llf - results.llf) * 2

    # Degrees of Freedom (df)
    dsf = results.df_modelwc - null_results.df_modelwc

    # P-value
    p_value = 1 - scipy.stats.chi2.sf(lrt, dsf)

    # Pseudo R-squared (pseudo_r2)
    pseudo_r2 = 1 - (
        np.exp(-2 * (results.llf - null_results.llf) / len(endog)) ** (2 / (len(endog) - len(exog_names) - 1))
    )

    # ICC
    icc = results.cov_re.iloc[0, 0] / (results.cov_re.iloc[0, 0] + results.scale) # Variance at level 2 (due to belonging to a certain job ad)/ Total variance

    # ICC null
    icc_null = null_results.cov_re.iloc[0, 0] / (null_results.cov_re.iloc[0, 0] + null_results.scale) # Level 2 variance/ Total variance

    # AIC
    aic = -2 * results.llf + 2 * np.log(results.nobs)

    # AIC null
    aic_null = -2 * null_results.llf + 2 * np.log(null_results.nobs)

    # BIC
    bic = -2 * results.llf + np.log(results.nobs) * (results.df_modelwc)

    # BIC null
    bic_null = -2 * null_results.llf + np.log(null_results.nobs) * (null_results.df_modelwc)

    return (
        lrt, dsf, p_value, pseudo_r2, icc, aic, bic,
        null_model, null_results, icc_null, aic_null, bic_null
    )


# Regressions

## Logistic Regression

In [11]:
def run_lg(df_name, df, ivs_type, ivs_name, ivs_, model_name=None, analysis_type=None):
    if model_name is None:
        model_name = 'Logistic'
    if analysis_type is None:
        analysis_type = 'regression'

    print('\n')
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {ivs_name} {"="*50}')
    for dv in tqdm.tqdm(dvs):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\nINDEPENDENT VARIABLE: {ivs_}\nCONTROLS: {controls[:1]}')
        print('\n')
        print('+'*120)

        endog_names = dv
        exog_names = [iv for iv in ivs_ if 'Mixed' not in ivs_] + controls[:1]

        endog = df[endog_names]
        exog = df[exog_names]
        constant = sm.add_constant(exog)

        model = sm.Logit(endog=endog, exog=constant, data=df)
        # formula = f'{dv} ~ {ivs_dummy[0]}*{ivs_dummy[3]} + {ivs_dummy[0]}*{ivs_dummy[4]} + {ivs_dummy[0]}*{ivs_dummy[5]} + {ivs_dummy[1]}*{ivs_dummy[3]} + {ivs_dummy[1]}*{ivs_dummy[4]} + {ivs_dummy[1]}*{ivs_dummy[5]} + {ivs_dummy[2]}*{ivs_dummy[3]} + {ivs_dummy[2]}*{ivs_dummy[4]} + {ivs_dummy[2]}*{ivs_dummy[5]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[1]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[4]} + {ivs_dummy[5]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[1]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[4]} + {ivs_dummy[5]}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]}'
        # formula = f'{dv} ~ {ivs_dummy[0]}*{ivs_dummy[3]} + {ivs_dummy[2]}*{ivs_dummy[5]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_dummy[0]}*{ivs_dummy[3]} + {ivs_dummy[2]}*{ivs_dummy[5]}'
        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]} + {ivs_dummy[0]}:{ivs_dummy[3]} + {ivs_dummy[2]}:{ivs_dummy[5]} + {controls_for_formula}'

        # formula = f'{dv} ~ {ivs_dummy[0]} + {ivs_dummy[2]} + {ivs_dummy[3]} + {ivs_dummy[5]} + {controls_for_formula}'

        # print('-'*20)
        # print(f'Using formula: {formula}')
        # print('-'*20)

        # # with contextlib.suppress(np.linalg.LinAlgError):
        # model = smf.logit(formula=formula, data=df)
        results = model.fit()
        full_summary = make_full_report(
            results, dv, dvs_name=dv, ivs_name=ivs_name, ivs_type=ivs_type, df_name=df_name, model_name=model_name, analysis_type=analysis_type
        )
        tt, df_std_coef = get_standardized_coefficients(results)
        print('\n')
        print('-'*20)
        print(f'{dv}\n')
        print('-'*20)
        print('\n')
        print(f'SUMMARY RESULTS:')
        print(results.summary())
        print(full_summary)
        print('\n')
        print('-'*20)
        print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        print('\n')
        print('-'*20)

        # # Display Results
        # print('~'*20)
        # print('+'*20)
        # print(f'{dv} x {ivs_}\n')
        # print('+'*20)
        # print('\n')
        # print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        # print('~'*20)
        # # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # # print('-'*20)
        # # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # # print('-'*20)
        # # print(f'COEFFICIENT:\n{results.params}')
        # # print('-'*20)
        # # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # # print(f'P-VALUES:\n{results.pvalues}')
        # # print('-'*20)
        # # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # # print(f'AIC:\n{results.aic:.2f}')
        # # print('-'*20)
        # # print(f'BIC:\n{results.bic:.2f}')
        # # print('-'*20)
        # # print(f'Coehn\'s F2:\n{results.prsquared:.5f}')
        # # print('-'*20)

        # save results
        save_name = f'{table_save_path}{model_name} {analysis_type} on {ivs_type} {df_name} - {dv} x {ivs_name}'
        results.save(f'{save_name}.pkl')
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
        df_summary_results.to_csv(f'{save_name}.csv')
        df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
        # df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
        # df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)
    return df_summary_results


In [12]:
%%time
if len(dataframes) > 1:
    @interact(df_name=dataframes.keys(), ivs_type=ivs_for_analysis.keys())
    def run_lg_interactive(df_name, ivs_type):
        df_lg_summary_results = run_lg(
            df_name=df_name,
            df=dataframes[df_name],
            ivs_type=ivs_type,
            ivs_name=ivs_for_analysis[ivs_type][0],
            ivs_=ivs_for_analysis[ivs_type][1],
        )
        display(df_lg_summary_results)
else:
    df_name = list(dataframes.keys())[0]
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]
    df_lg_summary_results = run_lg(
        df_name=df_name,
        df=dataframes[df_name],
        ivs_type=ivs_type,
        ivs_name=ivs_name,
        ivs_=ivs_,
    )
    display(df_lg_summary_results)




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs Categorical and PPS Gender and Age ==================================================


  0%|          | 0/2 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


DEPENDENT VARIABLE: Warmth
INDEPENDENT VARIABLE: ['Gender_Female', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector']
CONTROLS: ['Job Description spacy_sentencized_num_words']


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Optimization terminated successfully.
         Current function value: 0.516010
         Iterations 7


  0%|          | 0/98 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:05<00:05,  5.69s/it]

...

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:09<00:00,  4.60s/it]



--------------------
Competence

--------------------


SUMMARY RESULTS:
                           Logit Regression Results                           
Dep. Variable:             Competence   No. Observations:               308583
Model:                          Logit   Df Residuals:                   308573
Method:                           MLE   Df Model:                            9
Date:                Mon, 20 Nov 2023   Pseudo R-squ.:                 0.07753
Time:                        05:32:39   Log-Likelihood:            -1.9696e+05
converged:                       True   LL-Null:                   -2.1351e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
const                                        -188.7049      6.942    

,0,1,2,3,4,5,6
0,Logit Regression Results...,None,None,None,None,None,None
1,Dep. Variable:,Competence,No. Observations:,308583,None,None,None
2,Model:,Logit,Df Residuals:,308573,None,None,None
3,Method:,MLE,Df Model:,9,None,None,None
4,Date:,Mon,20 Nov 2023,Pseudo R-squ.:,0.07753,None,None
5,Time:,05:32:39,Log-Likelihood:,-1.9696e+05,None,None,None
6,converged:,True,LL-Null:,-2.1351e+05,None,None,None
7,Covariance Type:,nonrobust,LLR p-value:,0.000,None,None,None
8,,coef,std err,z,P>|z|,[0.025,0.975]
9,const,-188.7049,6.942,-27.185,0.000,-202.310,-175.100


CPU times: user 39.5 s, sys: 1.48 s, total: 41 s
Wall time: 9.23 s


## OLS Regression

In [13]:
def run_ols(df_name, df, ivs_type, ivs_name, ivs_, model_name=None, analysis_type=None):
    if model_name is None:
        model_name = 'OLS'
    if analysis_type is None:
        analysis_type = 'regression'

    if df_name == 'df_jobs':
        dvs_ = dvs_prob
    elif df_name == 'df_manual':
        dvs_ = dvs
    else:
        raise NameError(f'Dataframe name {df_name} name not in approved list.')

    print('\n')
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs_):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dvs_}\nINDEPENDENT VARIABLE: {ivs_}\nCONTROLS: {controls[:1]}')
        print('\n')
        print('+'*120)

        endog_names = dv
        exog_names = ivs_[:] + controls[:1]

        endog = df[endog_names]
        exog = df[exog_names]
        constant = sm.add_constant(exog)

        model = sm.OLS(endog=endog, exog=constant, data=df)
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {controls_for_formula}'

        # print('-'*20)
        # print(f'Using formula: {formula}')
        # print('-'*20)

        # model = smf.ols(formula=formula, data=df)
        # results = model.fit_regularized(alpha=0.0, L1_wt=0.0, start_params=None, profile_scale=False, refit=False)
        results = model.fit()
        full_summary = make_full_report(
            results, dv, dvs_name=dv, ivs_name=ivs_name, ivs_type=ivs_type, df_name=df_name, model_name=model_name, analysis_type=analysis_type
        )
        tt, df_std_coef = get_standardized_coefficients(results)
        print('\n')
        print('-'*20)
        print(f'{dv}\n')
        print('-'*20)
        print('\n')
        print('SUMMARY RESULTS:')
        print(results.summary())
        print(full_summary)
        print('\n')
        print('-'*20)
        print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        print('\n')
        print('-'*20)

        # # Display Results
        # print('~'*20)
        # print('+'*20)
        # print(f'{dv} x {ivs_[:] + controls[:1]}\n')
        # print('+'*20)
        # print('\n')
        # print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        # print('~'*20)
        # print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        # print('~'*20)
        # # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # # print('-'*20)
        # # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # # print('-'*20)
        # # print(f'COEFFICIENT:\n{results.params}')
        # # print('-'*20)
        # # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # # print(f'P-VALUES:\n{results.pvalues}')
        # # print('-'*20)
        # # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # # print(f'Coehn\'s F2:\n{results.rsquared_adj:.5f}')
        # # print('-'*20)
        # # table = sm.stats.anova_lm(results, typ=2)
        # # print(f'ANOVA:\n{table}')
        # # print('-'*20)

        # save results
        save_name = f'{table_save_path}{model_name} {analysis_type} on {ivs_type} {df_name} - {dv} x {ivs_name}'
        results.save(f'{save_name}.pkl')
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
        df_summary_results.to_csv(f'{save_name}.csv')
        df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
        df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
        df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)

        # # Boxplot
        # boxplot = df.boxplot([dv], by = [ivs_perc_[2], ivs_perc_[0]],
        #                     figsize = (16, 9),
        #                     showmeans = True,
        #                     notch = True)

        # boxplot.set_xlabel('Categories')
        # boxplot.set_ylabel(dv)
        # # Creating a path to save the plot.
        # plt.show()
        # plt.pause(.001)
        # # for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
        # #     save_path = f'{plot_save_path}Probability Boxplot - {df_name} - {dv} x Social Category Percentages.{image_save_format}'
        # #     boxplot.figure.savefig(
        # #         save_path, format=image_save_format,
        # #     )
        # plt.close()
    return df_summary_results, df_std_coef


In [14]:
%%time
if len(dataframes) > 1:
    @interact(df_name=dataframes.keys(), ivs_type=ivs_for_analysis.keys())
    def run_ols_interactive(df_name, ivs_type):
        df_ols_summary_results, df_ols_std_coef = run_ols(
            df_name=df_name,
            df=dataframes[df_name],
            ivs_type=ivs_type,
            ivs_name=ivs_for_analysis[ivs_type][0],
            ivs_=ivs_for_analysis[ivs_type][1],
        )
        display(df_ols_summary_results)
        display(df_ols_std_coef)
else:
    df_name = list(dataframes.keys())[0]
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]
    df_ols_summary_results, df_ols_std_coef = run_ols(
        df_name=df_name,
        df=dataframes[df_name],
        ivs_type=ivs_type,
        ivs_name=ivs_name,
        ivs_=ivs_,
    )
    display(df_ols_summary_results)
    display(df_ols_std_coef)




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================


  0%|          | 0/2 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


DEPENDENT VARIABLE: ['Warmth_Probability', 'Competence_Probability']
INDEPENDENT VARIABLE: ['Gender_Female', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector']
CONTROLS: ['Job Description spacy_sentencized_num_words']


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


  0%|          | 0/102 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:00<00:00,  1.52it/s]

...

100%|██████████| 2/2 [00:01<00:00,  1.47it/s]



--------------------
Competence_Probability

--------------------


SUMMARY RESULTS:
                              OLS Regression Results                              
Dep. Variable:     Competence_Probability   R-squared:                       0.105
Model:                                OLS   Adj. R-squared:                  0.105
Method:                     Least Squares   F-statistic:                     4042.
Date:                    Mon, 20 Nov 2023   Prob (F-statistic):               0.00
Time:                            05:32:40   Log-Likelihood:            -1.2058e+05
No. Observations:                  308583   AIC:                         2.412e+05
Df Residuals:                      308573   BIC:                         2.413e+05
Df Model:                               9                                         
Covariance Type:                nonrobust                                         
                                                  coef    std err          t      P

,0,1,2,3,4,5,6
0,OLS Regression Resul...,None,None,None,None,None,None
1,Dep. Variable:,Competence_Probability,R-squared:,0.105,None,None,None
2,Model:,OLS,Adj. R-squared:,0.105,None,None,None
3,Method:,Least Squares,F-statistic:,4042.,None,None,None
4,Date:,Mon,20 Nov 2023,Prob (F-statistic):,0.00,None,None
5,Time:,05:32:40,Log-Likelihood:,-1.2058e+05,None,None,None
6,No. Observations:,308583,AIC:,2.412e+05,None,None,None
7,Df Residuals:,308573,BIC:,2.413e+05,None,None,None
8,Df Model:,9,,,None,None,None
9,Covariance Type:,nonrobust,,,None,None,None


,Variable,Unstandardized Coefficent B (b),Standard Error,Standardized Coefficient b* (β),t-value,p-value,95% CI Lower,95% CI Upper
0,const,-32.43888,1.16839,-inf,-27.76376,0.00000,-34.72889,-30.14887
1,Gender_Female,-0.05811,0.00475,-0.13352,-12.23124,0.00000,-0.06742,-0.04880
2,Gender_Male,-0.00353,0.00326,-0.00733,-1.08175,0.27937,-0.00993,0.00287
3,Gender_Female_% per Sector,0.31587,0.01180,0.01622,26.77367,0.00000,0.29275,0.33900
4,Gender_Male_% per Sector,0.31506,0.01170,0.01615,26.93472,0.00000,0.29213,0.33798
5,Age_Older,0.01152,0.00334,0.02861,3.45307,0.00055,0.00498,0.01806
6,Age_Younger,-0.02534,0.00269,-0.07008,-9.43395,0.00000,-0.03060,-0.02007
7,Age_Older_% per Sector,0.01216,0.00102,0.00120,11.95028,0.00000,0.01017,0.01416
8,Age_Younger_% per Sector,0.01274,0.00099,0.00128,12.88172,0.00000,0.01080,0.01467
9,Job Description spacy_sentencized_num_words,0.00732,0.00004,0.00045,186.18248,0.00000,0.00724,0.00739


CPU times: user 4.56 s, sys: 336 ms, total: 4.9 s
Wall time: 1.38 s


## Interaction/Moderation OLS Regression

In [15]:
def run_mod_ols(df_name, df, ivs_type, ivs_name, ivs_, model_name=None, analysis_type=None):
    if model_name is None:
        model_name = 'Moderation OLS'
    if analysis_type is None:
        analysis_type = 'regression'

    if df_name == 'df_jobs':
        dvs_ = dvs_prob
    elif df_name == 'df_manual':
        dvs_ = dvs
    else:
        raise NameError(f'Dataframe name {df_name} name not in approved list.')

    print('\n')
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs_):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dvs_}\nINDEPENDENT VARIABLE: {ivs_}\nCONTROLS: {controls[:1]}')
        print('\n')
        print('+'*120)

        endog_names = dv
        exog_names = ivs_perc_and_perc_interactions[:] + controls[:1]

        endog = df[endog_names]
        exog = df[exog_names]
        constant = sm.add_constant(exog)

        model = sm.OLS(endog=endog, exog=constant, data=df)
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_interactions_[0]} + {ivs_perc_interactions_[1]} + {ivs_perc_interactions_[2]} +{ivs_perc_interactions_[3]} + {controls_for_formula}'

        # print('-'*20)
        # print(f'Using formula: {formula}')
        # print('-'*20)

        # model = smf.ols(formula=formula, data=df)
        results = model.fit()
        tt, df_std_coef = get_standardized_coefficients(results)
        print('\n')
        print('-'*20)
        print(f'{dv}\n')
        print('-'*20)
        print('\n')
        print('SUMMARY RESULTS:')
        print(results.summary())
        print('\n')
        print('-'*20)
        print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        print('\n')
        print('-'*20)

        # # Display Results
        # print('~'*20)
        # print('+'*20)
        # print(f'{dv} x {ivs_[:] + controls[:1]}\n')
        # print('+'*20)
        # print('\n')
        # print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        # print('~'*20)
        # print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        # print('~'*20)
        # # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # # print('-'*20)
        # # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # # print('-'*20)
        # # print(f'COEFFICIENT:\n{results.params}')
        # # print('-'*20)
        # # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # # print(f'P-VALUES:\n{results.pvalues}')
        # # print('-'*20)
        # # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # # print(f'Coehn\'s F2:\n{results.rsquared_adj:.5f}')
        # # print('-'*20)
        # # table = sm.stats.anova_lm(results, typ=2)
        # # print(f'ANOVA:\n{table}')
        # # print('-'*20)

        # save results
        save_name = f'{table_save_path}{model_name} {analysis_type} on {ivs_type} {df_name} - {dv} x {ivs_name}'
        results.save(f'{save_name}.pkl')
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
        df_summary_results.to_csv(f'{save_name}.csv')
        df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
        df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
        df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)

        # # Boxplot
        # boxplot = df.boxplot([dv], by = [ivs_perc_[2], ivs_perc_[0]],
        #                     figsize = (16, 9),
        #                     showmeans = True,
        #                     notch = True)

        # boxplot.set_xlabel('Categories')
        # boxplot.set_ylabel(dv)
        # # Creating a path to save the plot.
        # plt.show()
        # plt.pause(.001)
        # # for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
        # #     save_path = f'{plot_save_path}Probability Boxplot - {df_name} - {dv} x Social Category Percentages.{image_save_format}'
        # #     boxplot.figure.savefig(
        # #         save_path, format=image_save_format,
        # #     )
        # plt.close()
    return df_summary_results, df_std_coef


In [16]:
%%time
if len(dataframes) > 1:
    @interact(df_name=dataframes.keys(), ivs_type=ivs_for_analysis.keys())
    def run_mod_ols_interactive(df_name, ivs_type):
        df_mod_ols_summary_results, df_mod_ols_std_coef = run_mod_ols(
            df_name=df_name,
            df=dataframes[df_name],
            ivs_type=ivs_type,
            ivs_name=ivs_for_analysis[ivs_type][0],
            ivs_=ivs_for_analysis[ivs_type][1],
        )
        display(df_mod_ols_summary_results)
        display(df_mod_ols_std_coef)
else:
    df_name = list(dataframes.keys())[0]
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]
    df_mod_ols_summary_results, df_mod_ols_std_coef = run_mod_ols(
        df_name=df_name,
        df=dataframes[df_name],
        ivs_type=ivs_type,
        ivs_name=ivs_name,
        ivs_=ivs_,
    )
    display(df_mod_ols_summary_results)
    display(df_mod_ols_std_coef)




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================


  0%|          | 0/2 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


DEPENDENT VARIABLE: ['Warmth_Probability', 'Competence_Probability']
INDEPENDENT VARIABLE: ['Gender_Female', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector']
CONTROLS: ['Job Description spacy_sentencized_num_words']


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


 50%|█████     | 1/2 [00:00<00:00,  2.21it/s]



--------------------
Warmth_Probability

--------------------


SUMMARY RESULTS:
                            OLS Regression Results                            
Dep. Variable:     Warmth_Probability   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     5212.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):               0.00
Time:                        05:32:41   Log-Likelihood:            -1.0609e+05
No. Observations:              308583   AIC:                         2.122e+05
Df Residuals:                  308573   BIC:                         2.123e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------

100%|██████████| 2/2 [00:00<00:00,  2.47it/s]



--------------------
Competence_Probability

--------------------


SUMMARY RESULTS:
                              OLS Regression Results                              
Dep. Variable:     Competence_Probability   R-squared:                       0.106
Model:                                OLS   Adj. R-squared:                  0.106
Method:                     Least Squares   F-statistic:                     4060.
Date:                    Mon, 20 Nov 2023   Prob (F-statistic):               0.00
Time:                            05:32:41   Log-Likelihood:            -1.2051e+05
No. Observations:                  308583   AIC:                         2.410e+05
Df Residuals:                      308573   BIC:                         2.411e+05
Df Model:                               9                                         
Covariance Type:                nonrobust                                         
                                                  coef    std err          t      P

,0,1,2,3,4,5,6
0,OLS Regression Resul...,None,None,None,None,None,None
1,Dep. Variable:,Competence_Probability,R-squared:,0.106,None,None,None
2,Model:,OLS,Adj. R-squared:,0.106,None,None,None
3,Method:,Least Squares,F-statistic:,4060.,None,None,None
4,Date:,Mon,20 Nov 2023,Prob (F-statistic):,0.00,None,None
5,Time:,05:32:41,Log-Likelihood:,-1.2051e+05,None,None,None
6,No. Observations:,308583,AIC:,2.410e+05,None,None,None
7,Df Residuals:,308573,BIC:,2.411e+05,None,None,None
8,Df Model:,9,,,None,None,None
9,Covariance Type:,nonrobust,,,None,None,None


,Variable,Unstandardized Coefficent B (b),Standard Error,Standardized Coefficient b* (β),t-value,p-value,95% CI Lower,95% CI Upper
0,const,492.97889,984.00588,inf,0.50099,0.61638,-1435.64477,2421.60254
1,Gender_Female_% per Sector,-4.98982,9.84093,-0.25615,-0.50705,0.61212,-24.27777,14.29813
2,Gender_Male_% per Sector,-4.92188,9.83992,-0.25229,-0.50020,0.61694,-24.20784,14.36408
3,Age_Older_% per Sector,-4.20274,9.95853,-0.41553,-0.42202,0.67301,-23.72118,15.31570
4,Age_Younger_% per Sector,-5.59554,9.84465,-0.56061,-0.56838,0.56977,-24.89077,13.69969
5,Interaction_Female_Older_% per Sector,0.04265,0.09959,0.00004,0.42826,0.66847,-0.15255,0.23785
6,Interaction_Female_Younger_% per Sector,0.05659,0.09846,0.00005,0.57480,0.56543,-0.13638,0.24956
7,Interaction_Male_Older_% per Sector,0.04198,0.09958,0.00004,0.42155,0.67335,-0.15320,0.23716
8,Interaction_Male_Younger_% per Sector,0.05591,0.09844,0.00005,0.56798,0.57005,-0.13703,0.24886
9,Job Description spacy_sentencized_num_words,0.00735,0.00004,0.00045,186.86441,0.00000,0.00727,0.00742


CPU times: user 3.15 s, sys: 435 ms, total: 3.58 s
Wall time: 866 ms


## Multivariate OLS Regression

In [17]:
def run_mvols(df_name, df, ivs_type, ivs_name, ivs_, model_name=None, analysis_type=None):
    if model_name is None:
        model_name = 'Multivariate OLS'
    if analysis_type is None:
        analysis_type = 'regression'

    if df_name == 'df_jobs':
        dvs_ = dvs_all
    elif df_name == 'df_manual':
        dvs_ = dvs
    else:
        raise NameError(f'Dataframe name {df_name} name not in approved list.')

    df, ivs_for_analysis, controls, controls_for_formula = change_vars_for_formula(df, print_enabled=False)
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]

    print('\n')
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    print('+'*120)
    print('\n')
    print(f'DEPENDENT VARIABLE: {dvs_}\nINDEPENDENT VARIABLE: {ivs_}\nCONTROLS: {controls[:1]}')
    print('\n')
    print('+'*120)

    # endog_names = dvs_
    # exog_names = ivs_[:] + controls[:1]

    # endog = df[endog_names]
    # exog = df[exog_names]
    # constant = sm.add_constant(exog)

    # model = statsmodels.multivariate.multivariate_ols._MultivariateOLS(endog=endog, exog=constant)
    # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_for_formula}'
    # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_for_formula}'
    # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]}'
    formula = f'{" + ".join(dvs_)} ~ {" + ".join(ivs_)} + {controls_for_formula}'

    model = statsmodels.multivariate.multivariate_ols._MultivariateOLS.from_formula(formula=formula, data=df)

    print('-'*20)
    print(f'Using formula: {formula}')
    print('-'*20)

    with contextlib.suppress(ValueError):
        # model = smf.ols(formula=formula, data=df)
        results = model.fit()
        full_summary = results.mv_test().summary()
        # full_summary = make_full_report(
        #     results, dv, dvs__name=dv, ivs_name=ivs_name, ivs_type=ivs_type, df_name=df_name, model_name=model_name, analysis_type=analysis_type
        # )
        # tt, df_std_coef = get_standardized_coefficients(results)
        print('\n')
        print('-'*20)
        print(f'{dvs_}\n')
        print('-'*20)
        print('\n')
        print('SUMMARY RESULTS:')
        # print(results.summary())
        print(full_summary)
        # print('\n')
        # print('-'*20)
        # print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        # print('\n')
        # print('-'*20)
        # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # print('-'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # print('-'*20)
        # print(f'COEFFICIENT:\n{results.params}')
        # print('-'*20)
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # print(f'P-VALUES:\n{results.pvalues}')
        # print('-'*20)
        # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print(f'AIC:\n{results.aic:.2f}')
        # print('-'*20)
        # print(f'BIC:\n{results.bic:.2f}')
        # print('-'*20)
        # print(f'Coehn\'s F2:\n{results.rsquared_adj:.5f}')
        # print('-'*20)
        # table = sm.stats.anova_lm(results, typ=2)
        # print(f'ANOVA:\n{table}')
        # print('-'*20)

        # save results
        save_name = f'{table_save_path}{model_name} {analysis_type} on {ivs_type} {df_name} - {dvs_} x {ivs_name}'
        df_summary_results = pd.concat(pd.read_html(results.mv_test().summary().as_html()), axis='index', ignore_index=True)
        df_summary_results.to_csv(f'{save_name}.csv')
        df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
        # df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
        # df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)

        # # Boxplot
        # boxplot = df.boxplot([dv], by = [ivs_perc_[2], ivs_perc_[0]],
        #                     figsize = (16, 9),
        #                     showmeans = True,
        #                     notch = True)

        # boxplot.set_xlabel('Categories')
        # boxplot.set_ylabel(dv)
        # # Creating a path to save the plot.
        # plt.show()
        # plt.pause(.001)
        # # for image_save_format in tqdm.tqdm(['eps', 'png', 'svg']):
        # #     save_path = f'{plot_save_path}Probability Boxplot - {df_name} - {dv} x Social Category Percentages.{image_save_format}'
        # #     boxplot.figure.savefig(
        # #         save_path, format=image_save_format,
        # #     )
        # plt.close()
    return df_summary_results


In [18]:
%%time
if len(dataframes) > 1:
    @interact(df_name=dataframes.keys(), ivs_type=ivs_for_analysis.keys())
    def run_mvols_interactive(df_name, ivs_type):
        df_mvols_summary_results = run_mvols(
            df_name=df_name,
            df=dataframes[df_name],
            ivs_type=ivs_type,
            ivs_name=ivs_for_analysis[ivs_type][0],
            ivs_=ivs_for_analysis[ivs_type][1],
        )
        display(df_mvols_summary_results)
else:
    df_name = list(dataframes.keys())[0]
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]
    df_mvols_summary_results = run_mvols(
        df_name=df_name,
        df=dataframes[df_name],
        ivs_type=ivs_type,
        ivs_name=ivs_name,
        ivs_=ivs_,
    )
    display(df_mvols_summary_results)




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


DEPENDENT VARIABLE: ['Warmth', 'Competence', 'Warmth_Probability', 'Competence_Probability']
INDEPENDENT VARIABLE: ['Gender_Female', 'Gender_Male', 'Gender_Female_percentage_per_Sector', 'Gender_Male_percentage_per_Sector', 'Age_Older', 'Age_Younger', 'Age_Older_percentage_per_Sector', 'Age_Younger_percentage_per_Sector', 'Interaction_Female_Older_percentage_per_Sector', 'Interaction_Female_Younger_percentage_per_Sector', 'Interaction_Male_Older_percentage_per_Sector', 'Interaction_Male_Younger_percentage_per_Sector']
CONTROLS: ['Job_Description_spacy_sentencized_num_words']


++++++++++++++++++++++++++++++++++++++++++++++++

,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Intercept,Value,Num DF,Den DF,F Value,Pr > F
2,NaN,Wilks' lambda,0.9989,4.0000,308566.0000,87.1777,0.0000
3,NaN,Pillai's trace,0.0011,4.0000,308566.0000,87.1777,0.0000
4,NaN,Hotelling-Lawley trace,0.0011,4.0000,308566.0000,87.1777,0.0000
5,NaN,Roy's greatest root,0.0011,4.0000,308566.0000,87.1777,0.0000
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Gender_Female,Value,Num DF,Den DF,F Value,Pr > F
8,NaN,Wilks' lambda,0.9988,4.0000,308566.0000,89.5863,0.0000
9,NaN,Pillai's trace,0.0012,4.0000,308566.0000,89.5863,0.0000


CPU times: user 3.07 s, sys: 943 ms, total: 4.01 s
Wall time: 2.61 s


## Multi-level OLS Regression

In [19]:
def run_ml_ols(df_name, df, ivs_type, ivs_name, ivs_, model_name=None, analysis_type=None, random_intercept_names=None):
    if model_name is None:
        model_name = 'Multilevel OLS'
    if analysis_type is None:
        analysis_type = 'regression'
    if random_intercept_names is None:
        random_intercept_names = 'Job ID'

    if df_name == 'df_manual':
        dvs_ = dvs
    elif df_name == 'df_jobs':
        dvs_ = dvs_prob
    else:
        raise NameError(f'Dataframe name {df_name} name not in approved list.')

    print('\n')
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    for dv in tqdm.tqdm(dvs_):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\nINDEPENDENT VARIABLE: {ivs_}\nCONTROLS: {controls[:1]}')
        print('\n')
        print('+'*120)

        endog_names = dv
        exog_names = ivs_[:] + controls[:1]
        endog = df[endog_names]
        exog = df[exog_names]
        constant = sm.add_constant(exog)
        groups = df[random_intercept_names]

        # Main model
        model = sm.MixedLM(endog=endog, exog=constant, groups=groups)
        results = model.fit()

        # Get fit statistics
        (
            lrt, dsf, p_value, pseudo_r2, icc, aic, bic, null_model, null_results, icc_null, aic_null, bic_null
        ) = get_multilevel_reg_data(
            results, endog, groups
        )

        # Get standardized beta regression coefficients
        full_summary = make_full_report(
            results, dv, dvs_name=dv, ivs_name=ivs_name, ivs_type=ivs_type, df_name=df_name, model_name=model_name, analysis_type=analysis_type, title=f'Multilevel: {dv} x {ivs_name}'
        )
        tt, df_std_coef = get_standardized_coefficients(results)
        print('\n')
        print('-'*20)
        print(f'{dv}\n')
        print('-'*20)
        print('\n')
        print('-'*20)
        print(f'SUMMARY RESULTS:\n{full_summary}')
        print('\n')
        print('-'*20)
        print(f'FIT STATISTICS:\nLRT: {lrt:.5f}\nDSF: {dsf:.5f}\nP-VALUE: {p_value:.5f}\nPSEUDO R2: {pseudo_r2:.5f}\nICC: {icc:.5f}\nAIC: {aic:.5f}\nBIC: {bic:.5f}\n')
        print('-'*20)
        print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        print('\n')
        print('-'*20)

        # formula = f'{dv} ~ ' + ' + '.join(exog_names)
        # model0 = smf.mixedlm(formula, data=df, groups=groups, exog_re=exog_names)
        # results0 = model0.fit()
        # cov_params = results0.cov_params()
        # exog_vc = sm_mlm.cov_struct.CovarianceStruct().from_params(cov_params.values, cov_type='custom')

        # model = sm.MixedLM(endog=endog, exog=constant, exog_re=exog_names, exog_vc=exog_vc, groups=groups)
        # results = model.fit()

        # endog = df[dv]
        # exog0 = df[['Intercept', f'{list(iter(ivs_for_analysis))[0]}']]
        # exog1 = df[['Intercept', f'{list(iter(ivs_for_analysis))[1]}']]
        # iv_1 = list(iter(ivs_for_analysis))[0]
        # iv_1_treatment = ivs_for_analysis[iv_1][0]
        # iv_2 = list(iter(ivs_for_analysis))[1]
        # iv_2_treatment = ivs_for_analysis[iv_2][0]

        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]} + {controls_for_formula}'
        # formula = f'{dv} ~ {ivs_perc_[0]}*{ivs_perc_[2]} + {ivs_perc_[0]}*{ivs_perc_[3]} + {ivs_perc_[1]}*{ivs_perc_[2]} + {ivs_perc_[1]}*{ivs_perc_[3]}'
        # formula = f'{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]}'

        # print('-'*20)
        # print(f'Using formula: {formula}')
        # print('-'*20)

        # vc_formula = {f'{controls_[1]}': f'0 + {controls_[1]}'}
        # re_formula = f'1 + {controls_[1]}'

        # model = smf.mixedlm(formula=formula, data=df, groups='Job_ID',) #vc_formula=vc_formula, re_formula=re_formula)
        # results = model.fit(method='lbfgs')
        # gradient = model.score(results.params_object)

        # # Display Results
        # print('~'*20)
        # print('+'*20)
        # print(f'{dv} x {ivs_}\n')
        # print('+'*20)
        # # print(f'Gradient:\n{gradient}')
        # # print('\n')
        # print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        # print('~'*20)
        # print(f'STANDARDIZED BETA REGRESSION COEFFICIENTS FOR {dv}:\n{df_std_coef}')
        # print('~'*20)
        # print(f'NULL MODEL SUMMARY:\n{null_results.summary()}')
        # print('\n')
        # print('+'*20)
        # # print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        # # print('-'*20)
        # # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        # # print('-'*20)
        # # print(f'COEFFICIENT:\n{results.params}')
        # # print('-'*20)
        # # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        # # print(f'P-VALUES:\n{results.pvalues}')
        # # print('-'*20)
        # # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        # print('+'*20)
        # print('-'*20)
        # print(f'Akaike Information Criterion (AIC):\n{aic}')
        # print('-'*20)
        # print(f'AIC NULL:\n{aic_null}')
        # print('-'*20)
        # print(f'Bayesian Information Criterion (BIC):\n{bic}')
        # print('-'*20)
        # print(f'BIC NULL:\n{bic_null}')
        # print('-'*20)
        # print(f'Intraclass Correlation Coefficient (ICC):\n{icc}')
        # print('-'*20)
        # print(f'ICC NULL:\n{icc_null}')
        # print('-'*20)
        # print(f'Pseudo R2:\n{pseudo_r2}')
        # print('-'*20)
        # print(f'Likelihood Ratio Test of random-effects (LRT):\n{lrt}')
        # print('-'*20)
        # print(f'Degrees of Freedom:\n{dsf}')
        # print('-'*20)
        # print(f'P-VALUE:\n{p_value}')
        # print('-'*20)
        # print('+'*20)
        # # print(f'Coehn\'s F2:\n{results.rsquared_adj:.5f}')
        # # print('-'*20)
        # # table = sm.stats.anova_lm(results, typ=2)
        # # print(f'ANOVA:\n{table}')
        # # print('-'*20)

        # save results
        save_name = f'{table_save_path}{model_name} {analysis_type} on {ivs_type} {df_name} - {dv} x {ivs_name}'
        df_summary_results = pd.concat(pd.read_html(results.summary().as_html()), axis='index', ignore_index=True)
        df_summary_results.to_csv(f'{save_name}.csv')
        df_summary_results.style.to_latex(f'{save_name}.tex', hrules=True)
        df_std_coef.to_csv(f'{save_name} - standardized coefficients.csv')
        df_std_coef.style.to_latex(f'{save_name} - standardized coefficients.tex', hrules=True)

        # # Normality Tests (https://www.pythonfordatascience.org/mixed-effects-regression-python/)
        # ## Residual and Kernal Density Estimate (KDE) Plot for Homoskedasticity
        # fig = plt.figure(figsize = (16, 9))

        # ax = sns.distplot(results.resid, hist = True, kde_kws = {"shade" : True, "lw": 1}, fit = scipy.stats.norm, kde=True, palette='colorblind')

        # ax.set_title(f"Kernal Density Estimate (KDE) Plot of Model Residuals (Blue) and Normal Distribution (Black)\n{save_name}")
        # ax.set_xlabel("Residuals")
        # fig.show('notebook')
        # plt.pause(.001)

        # # Q-Q Plot
        # fig = plt.figure(figsize = (16, 9))
        # ax = fig.add_subplot(111)

        # qq = sm.qqplot(results.resid, dist = scipy.stats.norm, line = 's', ax = ax, color='blue', markerfacecolor='blue')
        # ax.set_title(f"Q-Q Plot\n{save_name}",fontsize=15)
        # ax.xaxis.get_label().set_fontsize(12)
        # ax.yaxis.get_label().set_fontsize(12)
        # ax.get_lines()[0].set_color('black')
        # ax.get_lines()[0].set_linewidth('2')
        # ax.get_lines()[1].set_color('black')
        # ax.get_lines()[1].set_linewidth('2')
        # fig.show('notebook')
        # plt.pause(.001)

        # # Test of Normality
        # norm = scipy.stats.normaltest(results.resid)

        # print('='*80)
        # print(f'{dv} Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Skewness-Kurtosis Test of Normality
        # norm_sk = scipy.stats.kurtosistest(results.resid)

        # print('='*80)
        # print(f'{dv} Skewness-Kurtosis Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Shapir-Wilk Test of Normality
        # norm_res = scipy.stats.shapiro(results.resid)

        # print('='*80)
        # print(f'{dv} Shapir-Wilk Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_res)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Anderson-Darling Test of Normality
        # norm_and = scipy.stats.anderson(results.resid)

        # print('='*80)
        # print(f'{dv} Anderson-Darling Test of Normality:')
        # print('-'*80)
        # for key, val in dict(zip(normality_tests_labels, norm_and)).items():
        #     print(key,': ', val) # Significant
        # print('\n')

        # # Residuals versus Fitted values (RVF) Plot for Homoskedasticity
        # fig = plt.figure(figsize = (16, 9))

        # ax = sns.scatterplot(y = results.resid, x = results.fittedvalues, palette='colorblind')

        # ax.set_title(f"Residuals versus Fitted values (RVF) Plot\n{save_name}")
        # ax.set_xlabel("Fitted Values")
        # ax.set_ylabel("Residuals")
        # fig.show('notebook')
        # plt.pause(.001)

        # # White’s Lagrange Multiplier Test for Heteroscedasticity
        # het_white_res = het_white(results.resid, results.model.exog)

        # het_white_labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]

        # print('='*80)
        # print('White’s Lagrange Multiplier Test for Heteroscedasticity')
        # print('-'*80)
        # for key, val in dict(zip(het_white_labels, het_white_res)).items():
        #     print(key, val)
        # print('\n')
        # print('\n')
        # print('+'*120)
        # print('\n')
    return df_summary_results, df_std_coef


In [20]:
%%time
if len(dataframes) > 1:
    @interact(df_name=dataframes.keys(), ivs_type=ivs_for_analysis.keys())
    def run_ml_ols_interactive(df_name, ivs_type):
        df_ml_ols_summary_results, df_ml_ols_std_coef = run_ml_ols(
            df_name=df_name,
            df=dataframes[df_name],
            ivs_type=ivs_type,
            ivs_name=ivs_for_analysis[ivs_type][0],
            ivs_=ivs_for_analysis[ivs_type][1],
        )
        display(df_ml_ols_summary_results)
        display(df_ml_ols_std_coef)
else:
    df_name = list(dataframes.keys())[0]
    ivs_type = list(ivs_for_analysis.keys())[0]
    ivs_name = ivs_for_analysis[ivs_type][0]
    ivs_ = ivs_for_analysis[ivs_type][1]
    df_ml_ols_summary_results, df_ml_ols_std_coef = run_ml_ols(
        df_name=df_name,
        df=dataframes[df_name],
        ivs_type=ivs_type,
        ivs_name=ivs_name,
        ivs_=ivs_,
    )
    display(df_ml_ols_summary_results)
    display(df_ml_ols_std_coef)




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================


  0%|          | 0/2 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


DEPENDENT VARIABLE: Warmth_Probability
INDEPENDENT VARIABLE: ['Gender_Female', 'Gender_Male', 'Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older', 'Age_Younger', 'Age_Older_% per Sector', 'Age_Younger_% per Sector']
CONTROLS: ['Job Description spacy_sentencized_num_words']


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


  0%|          | 0/102 [00:00<?, ?it/s]

 50%|█████     | 1/2 [13:13<13:13, 793.96s/it]

...

100%|██████████| 2/2 [32:51<00:00, 985.65s/it] 



--------------------
Competence_Probability

--------------------


--------------------
SUMMARY RESULTS:
Multilevel: Competence_Probability x Categorical and PPS Gender and Age
                                               Competence  
-----------------------------------------------------------
const                                       -36.19***      
                                            (2.33)         
Gender_Female                               -0.05***       
                                            (0.01)         
Gender_Male                                 0.01           
                                            (0.01)         
Gender_Female_% per Sector                  0.35***        
                                            (0.02)         
Gender_Male_% per Sector                    0.35***        
                                            (0.02)         
Age_Older                                   -0.00          
                                        

,0,1,2,3,4,5,6
0,Model:,MixedLM,Dependent Variable:,Competence_Probability,NaN,NaN,NaN
1,No. Observations:,308583,Method:,REML,NaN,NaN,NaN
2,No. Groups:,16135,Scale:,0.1115,NaN,NaN,NaN
3,Min. group size:,1,Log-Likelihood:,-108748.0093,NaN,NaN,NaN
4,Max. group size:,277,Converged:,Yes,NaN,NaN,NaN
5,Mean group size:,19.1,NaN,NaN,NaN,NaN,NaN
6,NaN,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
7,const,-36.194,2.333,-15.515,0.000,-40.766,-31.622
8,Gender_Female,-0.053,0.010,-5.599,0.000,-0.072,-0.035
9,Gender_Male,0.007,0.007,1.047,0.295,-0.006,0.020


,Variable,Unstandardized Coefficent B (b),Standard Error,Standardized Coefficient b* (β),t-value,p-value,95% CI Lower,95% CI Upper
0,const,-36.19409,2.33291,-inf,-15.51459,0.00000,-40.76650,-31.62167
1,Gender_Female,-0.05334,0.00953,-0.12255,-5.59872,0.00000,-0.07201,-0.03466
2,Gender_Male,0.00694,0.00663,0.01442,1.04708,0.29506,-0.00605,0.01994
3,Gender_Female_% per Sector,0.34918,0.02365,0.01793,14.76179,0.00000,0.30282,0.39554
4,Gender_Male_% per Sector,0.34845,0.02345,0.01786,14.86196,0.00000,0.30250,0.39440
5,Age_Older,-0.00313,0.00636,-0.00778,-0.49242,0.62242,-0.01560,0.00933
6,Age_Younger,-0.01961,0.00556,-0.05425,-3.52678,0.00042,-0.03051,-0.00871
7,Age_Older_% per Sector,0.01602,0.00207,0.00158,7.72243,0.00000,0.01195,0.02008
8,Age_Younger_% per Sector,0.01649,0.00201,0.00165,8.19374,0.00000,0.01255,0.02044
9,Job Description spacy_sentencized_num_words,0.00845,0.00004,0.00051,210.45028,0.00000,0.00837,0.00853


CPU times: user 3h 6min 13s, sys: 14min 46s, total: 3h 21min
Wall time: 32min 51s
